<a href="https://colab.research.google.com/github/Agrannya-Singh/othello_api/blob/main/reversi_agent_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%writefile game_logic.py

import numpy as np
import random

# Constants
BLACK = 1
WHITE = -1
EMPTY = 0
BOARD_SIZE = 8
DIRECTIONS = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]

class Othello:
    """Handles all Othello game logic for the ML agent."""
    def __init__(self, model=None):
        self.board = np.zeros((BOARD_SIZE, BOARD_SIZE), dtype=int)
        self.board[3][3] = WHITE
        self.board[3][4] = BLACK
        self.board[4][3] = BLACK
        self.board[4][4] = WHITE
        self.current_player = BLACK
        self.model = model

    def make_move(self, move):
        if move is None or not self.is_valid_move(move[0], move[1]):
            return False
        row, col = move
        self.board[row][col] = self.current_player
        for dr, dc in DIRECTIONS:
            r, c = row + dr, col + dc
            to_flip = []
            while 0 <= r < BOARD_SIZE and 0 <= c < BOARD_SIZE and self.board[r][c] == -self.current_player:
                to_flip.append((r, c))
                r += dr
                c += dc
            if 0 <= r < BOARD_SIZE and 0 <= c < BOARD_SIZE and self.board[r][c] == self.current_player:
                for fr, fc in to_flip:
                    self.board[fr][fc] = self.current_player
        return True

    def get_valid_moves(self):
        valid_moves = []
        for r in range(BOARD_SIZE):
            for c in range(BOARD_SIZE):
                if self.is_valid_move(r, c):
                    valid_moves.append((r, c))
        return valid_moves

    def is_valid_move(self, row, col):
        if self.board[row][col] != EMPTY:
            return False
        for dr, dc in DIRECTIONS:
            r, c = row + dr, col + dc
            if 0 <= r < BOARD_SIZE and 0 <= c < BOARD_SIZE and self.board[r][c] == -self.current_player:
                r += dr
                c += dc
                while 0 <= r < BOARD_SIZE and 0 <= c < BOARD_SIZE:
                    if self.board[r][c] == self.current_player: return True
                    if self.board[r][c] == EMPTY: break
                    r += dr
                    c += dc
        return False

    def simulate_move(self, board, player, move):
        row, col = move
        new_board = board.copy()
        new_board[row][col] = player
        for dr, dc in DIRECTIONS:
            r, c = row + dr, col + dc
            to_flip = []
            while 0 <= r < BOARD_SIZE and 0 <= c < BOARD_SIZE and new_board[r][c] == -player:
                to_flip.append((r, c))
                r += dr
                c += dc
            if 0 <= r < BOARD_SIZE and 0 <= c < BOARD_SIZE and new_board[r][c] == player:
                for fr, fc in to_flip:
                    new_board[fr][fc] = player
        return new_board

    def get_best_move_ml(self):
        valid_moves = self.get_valid_moves()
        if not valid_moves: return None
        if self.model is None: return random.choice(valid_moves)

        temp_boards = []
        for move in valid_moves:
            temp_board = self.simulate_move(self.board, self.current_player, move)
            temp_boards.append(temp_board.flatten())

        if temp_boards:
            batch = np.vstack(temp_boards)
            scores = self.model.predict(batch, verbose=0).flatten()
        else:
            scores = []

        if self.current_player == BLACK:
            best_idx = np.argmax(scores)
        else:
            best_idx = np.argmin(scores)

        return valid_moves[best_idx]

    def is_game_over(self):
        if self.get_valid_moves(): return False
        self.current_player *= -1
        has_moves = bool(self.get_valid_moves())
        self.current_player *= -1
        return not has_moves

    def get_winner(self):
        black_count = np.sum(self.board == BLACK)
        white_count = np.sum(self.board == WHITE)
        if black_count > white_count: return BLACK
        if white_count > black_count: return WHITE
        return EMPTY

In [ ]:
#creating initial model
from tensorflow.keras import models, layers, losses

BOARD_SIZE = 8
MODEL_PATH = 'othello_model.h5'

print("Creating a new, untrained model blueprint...")
model = models.Sequential([
    layers.Dense(128, activation='relu', input_shape=(BOARD_SIZE * BOARD_SIZE,)),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='tanh')
])
model.compile(optimizer='adam', loss=losses.MeanSquaredError())
model.save(MODEL_PATH)
print(f"Success! Initial model saved to '{MODEL_PATH}'.")

In [ ]:
# train.py

from game_logic import Othello, BLACK
from tensorflow.keras import models, losses
import numpy as np
from google.colab import files

MODEL_PATH = 'othello_model.h5'

def run_training_session(num_games=2000, epochs=15):
    print(f"Starting self-play training session for {num_games} games...")
    model = models.load_model(MODEL_PATH)
    # Re-compile to ensure compatibility with the Colab environment
    model.compile(optimizer='adam', loss=losses.MeanSquaredError())

    all_game_data = []

    for i in range(num_games):
        game = Othello(model=model)
        game_states = []  # List of (board_vector, current_player)

        while not game.is_game_over():
            # Store state BEFORE the move
            game_states.append((game.board.flatten(), game.current_player))
            move = game.get_best_move_ml()
            if move is None:
                game.current_player *= -1
                continue
            game.make_move(move)
            game.current_player *= -1

        outcome = game.get_winner()
        for board_vector, player in game_states:
            # The label is the final game outcome, from that player's perspective
            if outcome == 0:
                score = 0.0
            else:
                score = 1.0 if player == outcome else -1.0
            all_game_data.append((board_vector, score))

        if (i + 1) % 100 == 0:
            print(f"  ...completed game {i+1}/{num_games}")

    X_train = np.array([data[0] for data in all_game_data])
    y_train = np.array([data[1] for data in all_game_data])

    print(f"\nTraining model on data from {num_games} games...")
    model.fit(X_train, y_train, epochs=epochs, batch_size=64, validation_split=0.1)

    model.save(MODEL_PATH)
    print(f"Training complete. New model saved to {MODEL_PATH}")

    # Automatically download the model
    files.download(MODEL_PATH)

if __name__ == '__main__':
    run_training_session()

Starting self-play training session for 2000 games...


In [ ]:
# evaluation script for the model:
# evaluate.py

import numpy as np
from tensorflow.keras import models
from game_logic import Othello, BLACK, WHITE, EMPTY

MODEL_PATH = 'othello_model.h5'

def minmax(board, depth, player, alpha, beta):
    game = Othello()  # Temp instance for helper functions
    game.board = board.copy()
    game.current_player = player

    if depth == 0 or game.is_game_over():
        black_count = np.sum(board == BLACK)
        white_count = np.sum(board == WHITE)
        return black_count - white_count if player == BLACK else white_count - black_count

    valid_moves = game.get_valid_moves()
    if not valid_moves:
        return minmax(board, depth - 1, -player, alpha, beta)  # Pass turn

    if player == BLACK:  # Maximizing
        max_eval = -np.inf
        for move in valid_moves:
            new_board = game.simulate_move(board, player, move)
            eval = minmax(new_board, depth - 1, -player, alpha, beta)
            max_eval = max(max_eval, eval)
            alpha = max(alpha, eval)
            if beta <= alpha:
                break
        return max_eval
    else:  # Minimizing
        min_eval = np.inf
        for move in valid_moves:
            new_board = game.simulate_move(board, player, move)
            eval = minmax(new_board, depth - 1, -player, alpha, beta)
            min_eval = min(min_eval, eval)
            beta = min(beta, eval)
            if beta <= alpha:
                break
        return min_eval

def get_best_move_minmax(game, depth=5):
    valid_moves = game.get_valid_moves()
    if not valid_moves: return None

    best_move = None
    best_value = -np.inf if game.current_player == BLACK else np.inf
    alpha = -np.inf
    beta = np.inf

    for move in valid_moves:
        new_board = game.simulate_move(game.board, game.current_player, move)
        value = minmax(new_board, depth - 1, -game.current_player, alpha, beta)
        if game.current_player == BLACK:
            if value > best_value:
                best_value = value
                best_move = move
            alpha = max(alpha, value)
        else:
            if value < best_value:
                best_value = value
                best_move = move
            beta = min(beta, value)
    return best_move

def evaluate_model(num_games=20, depth=5):
    model = models.load_model(MODEL_PATH)
    wins = 0
    draws = 0
    losses = 0

    for i in range(num_games):
        # Alternate starting player: even games ML as BLACK, odd as WHITE
        ml_player = BLACK if i % 2 == 0 else WHITE
        minmax_player = -ml_player

        game = Othello(model=model)
        game.current_player = BLACK  # Always start with BLACK

        while not game.is_game_over():
            if game.current_player == ml_player:
                move = game.get_best_move_ml()
            else:
                move = get_best_move_minmax(game, depth=depth)
            if move is None:
                game.current_player *= -1
                continue
            game.make_move(move)
            game.current_player *= -1

        winner = game.get_winner()
        if winner == ml_player:
            wins += 1
        elif winner == minmax_player:
            losses += 1
        else:
            draws += 1

        print(f"Game {i+1}: ML as {ml_player} - Winner: {winner} (Wins: {wins}, Draws: {draws}, Losses: {losses})")

    win_rate = (wins / num_games) * 100
    print(f"\nFinal Win Rate: {win_rate}% (Wins: {wins}, Draws: {draws}, Losses: {losses})")
    return win_rate >= 80

if __name__ == '__main__':
    evaluate_model()